# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import sympy as smp

In [ ]:
#| export
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

### Data

Properties Data

In [ ]:
class Object:
    pass

In [ ]:
#| export
class PropertyData(dict):
    pass

In [ ]:
c = PropertyData({'0': {'mass': 2, 'object': Object()}})

In [ ]:
c['0']

{'mass': 2, 'object': <__main__.Object>}

In [ ]:
c['1'] = {'mass': 2.1, 'object': 'XXX'}

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(c)

In [ ]:
df

,0,1
mass,2,2.1
object,<__main__.Object object at 0x7fc720c82b50>,XXX


### Proprety

In [ ]:
class Bookshelf:
    def __init__(self):
        print(self.x)

In [ ]:
class Book(Bookshelf):
    def __init__(self):
        self.x = 1
        super().__init__()

In [ ]:
b = Book()

1


In [ ]:
#| export
class PropertyObservable:
    def __init__(self, compound):
        self._data = PropertyData()
        self.compound = compound
        self._connections = []
        self._laws = []
        self.func_changed = Event()
        
        self.symbol = smp.symbols(f'{self.abbreviate}', real=True)
    
    def add_data(self, time, value):
        self._data[time] = {'value': value}
    
    def add_law(self, law):
        print(f'about to add law {law}')
        if not law in self._laws:
            self._laws.append(law)

In [ ]:
#| export
class Mass(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'm'
        super().__init__(compound)

In [ ]:
#| export
class Mole(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'n'
        super().__init__(compound)

In [ ]:
#| export
class Pressure(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'P'
        super().__init__(compound)

In [ ]:
#| export
class Volume(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'V'
        super().__init__(compound)

In [ ]:
#| export
class Temperature(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'T'
        super().__init__(compound)

### Law

In [ ]:
#| export
class Law:
    def init_properties(self):
        for p in self._properties:
            name = p.__name__.lower()
            print(f'####### initialzing {name} #####')
            
            if not name in self.compound._properties:
                self.compound._properties[name] = p(compound=self.compound)
            
            print(f'add to {self}')
            self.compound._properties[name].add_law(self)

### Mediator

In [ ]:
#| export
class Mediator:
    pass